<a href="https://colab.research.google.com/github/orlandoalexander/Moda-Personal-Assistant/blob/preproc_package/ModelTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Input
import pandas as pd
import numpy as np


In [ ]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install git+https://github.com/orlandoalexander/Moda-Personal-Assistant.git@preproc_package


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/orlandoalexander/Moda-Personal-Assistant.git (to revision preproc_package) to /tmp/pip-req-build-4swkp6f8
  Running command git clone --filter=blob:none --quiet https://github.com/orlandoalexander/Moda-Personal-Assistant.git /tmp/pip-req-build-4swkp6f8
  Running command git checkout -b preproc_package --track origin/preproc_package
  Switched to a new branch 'preproc_package'
  Branch 'preproc_package' set up to track remote branch 'preproc_package' from 'origin'.
  Resolved https://github.com/orlandoalexander/Moda-Personal-Assistant.git to commit 9063ecadadef2e41685d319596a2622d190bbb74
  Preparing metadata (setup.py) ... done


In [ ]:
# !pip install -r requirements.txt

In [ ]:
# !unzip '/content/drive/MyDrive/attribute_img/img.zip' -d '/content/Moda-Personal-Assistant'


In [ ]:
# !pip install -U pillow

In [ ]:
from preproc.attributes import AttributePreproc


In [ ]:
# !unzip '/content/drive/MyDrive/attribute_img/img.zip'

In [ ]:
prep = AttributePreproc('img', 
                        (224,224), 
                        'sleeves',
                        0.2) 
# e.g. AttributePreproc(folder/img, (256,256), 'fabric', 0.2)



In [ ]:
X_train, X_test, y_train, y_test  = prep.run()

Augmenting attribute 'long_sleeve'...
Augmenting attribute 'short_sleeve'...
Augmenting attribute 'sleeveless'...
Done!


/usr/local/lib/python3.8/dist-packages/preproc/attributes.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self._y_train, self._y_test = np.array(list(self._y_train)), np.array(list(self._y_test))


In [ ]:
from keras.applications import mobilenet


In [ ]:
X_train = mobilenet.preprocess_input(X_train)

In [ ]:
X_test = mobilenet.preprocess_input(X_test)

ValueError: ignored

In [ ]:
X_test

In [ ]:
y_train.columns


Index(['long_sleeve', 'short_sleeve', 'sleeveless'], dtype='object')

In [ ]:
for i in dir(preproc_obj):
    del i
del preproc_obj

In [ ]:
import gc

In [ ]:

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D
from keras import Sequential
from keras.metrics import Precision, Recall
from keras.optimizers import Adam

from keras.applications import InceptionV3
from keras.applications import inception_v3
from keras.applications import ResNet50
from keras.applications import resnet
from keras.applications import MobileNetV2
from keras.applications import mobilenet_v2
from keras.applications import EfficientNetB0
from keras.applications import efficientnet

# attribute should be one of the following strings:
# 'design', 'sleeves', 'length', 'neckline', 'fabric', 'fit'

# model should be one of the following strings:
# 'inception', 'resnet', 'mobilenet', 'efficientnet'

class AttrModel:
    def __init__(self, attribute, epochs, model, input_shape, batch_size, final_layer_neurons,
                 X_train, X_test,
                 y_train, y_test, **kwargs):
        self.attribute = attribute
        self.model = model.lower()
        self.input_shape = input_shape
        self.batch_size = batch_size
        self.epochs = epochs
        self.final_layer_neurons = final_layer_neurons
        self.kwargs = kwargs
        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            X_train, y_train, test_size=.5
        )
        self.X_test = X_test
        self.y_test = y_test
        del X_train, X_val, X_test, y_train, y_val, y_test
        gc.collect()
        self.cat_nums = {
            'design': 7,
            'sleeves': 3,
            'length': 2,
            'neckline': 4,
            'fabric': 6,
            'fit': 3
        }
        self.cat_num = self.cat_nums[self.attribute]
        self.activation = 'sigmoid' if self.attribute == 'length' else 'softmax'
        self.loss = 'binary_crossentropy' if self.attribute == 'length' else 'categorical_crossentropy'
        self.model = self.instantiate_model()  # calling the function below


    def instantiate_model(self):
        # import ipdb;
        # ipdb.set_trace()
        input = Input(self.input_shape)
        if self.model == 'inception':              # calling the chosen pretrained model
            base_model = InceptionV3(include_top=False, weights='imagenet',
                                    classes=self.cat_num, input_tensor=input)
        elif self.model == 'resnet':
            base_model = ResNet50(include_top=False, weights='imagenet',
                                  classes=self.cat_num, input_tensore=input)
        elif self.model == 'mobilenet':
            base_model = MobileNetV2(include_top=False, weights='imagenet',
                                     classes=self.cat_num, input_tensor=input)
        elif self.model == 'efficientnet':
            base_model = EfficientNetB0(include_top=False, weights='imagenet',
                                        classes=self.cat_num, input_tensor=input)
        else:
            print('''No model found. Please pass one of the following:
                  inception, resnet, mobilenet, efficientnet''')

        base_model.trainable = False    # freeze layers

        pool = GlobalAveragePooling2D()
        dense = Dense(units=self.final_layer_neurons, activation='relu')
        prediction = Dense(units=self.cat_num, activation=self.activation)
        model = Sequential([base_model, pool, dense, prediction])
        print(self.loss) # check loss func
        model.compile(loss=self.loss, optimizer='adam',
                      metrics=['accuracy', Precision(), Recall()])

        return model

    def train(self):
        # import ipdb
        # ipdb.set_trace()
        self.model.history = self.model.fit(
            self.X_train, self.y_train,
            epochs=self.epochs, batch_size=self.batch_size,
            validation_split=0.2, verbose=1, 
            callbacks=[EarlyStopping(patience=2, restore_best_weights=True)])
        return self.model.history

    def finetune(self,callbacks=[EarlyStopping(patience=3)],
                 loss='categorical_crossentropy',
                 metrics=['accuracy', Precision(), Recall()]):
        self.model.trainable = True     # unfreeze layers, then compile to save changes
        self.model.compile(
            optimizer=Adam(1e-5),                  # Very low learning rate
            loss=loss,
            metrics=metrics)
        self.model.fit(self.X_val, self.y_val,
                       epochs=self.epochs, batch_size=self.batch_size,
                       callbacks=[EarlyStopping(patience=2, monitor='val_accuracy', 
                                                restore_best_weights=True)],
                       validation_split=0.2)
        return self.model.history

    def evaluate(self):
        return self.model.evaluate(self.X_test, self.y_test, verbose=1)

    def predict(self, X):
        return self.model.predict(X)


In [ ]:
inception = AttrModel(attribute='neckline',
                      model='mobilenet',
                      input_shape=(224,224,3),
                      batch_size= 16,
                      epochs=50,
                      final_layer_neurons=100,
                      X_train=X_train,
                      X_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

In [ ]:
inception.train()

In [ ]:
inception.finetune()

In [ ]:
inception.evaluate()

# New Section